In [1]:
#get document frequency for terms in the same article/instance; and get each sentence's BOW; and get each question's BOW;
#get each sentence's length; get each article's average length of sentences
import csv
import json
import copy
import os
import nltk
from math import log
from collections import defaultdict, Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial.distance import cosine as cos_distance
from sklearn.decomposition import TruncatedSVD
from nltk.tag import StanfordNERTagger
def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma
def input_data():
    base_path = os.path.join('data/')
    train_file = base_path + 'QA_train.json'
    train_data = json.load(open(train_file))
    test_file = base_path + 'QA_test.json'
    test_data = json.load(open(test_file))
    dev_file = base_path + 'QA_dev.json'
    dev_data = json.load(open(dev_file))

    return train_data,test_data,dev_data
def transform_text(text):
    text = nltk.word_tokenize(text)
    text = [lemmatize(word.lower()) for word in text]
    result = []
    for word in text:
        if word not in stopwords and word not in punctuations:
            result.append(word)
    return result
def get_Docfrequency_SentenceBOW(dataset):
    #save dics, each dictionary contains document frequencies for all terms in the same article
    question_list = []
    #save lists, each list represent an article, saving sentences' bow
    total_sentence_bow = []
    #save lists, each list represent an article, saving questions' bow
    total_question_bow = []
    #save lists, each list represent all sentences' lengthes.
    sent_lengthes = []
    #save a list, each item represents the average length of sentences
    avg_lengthes = []
    #
    answer_id = []
    
    for article in dataset:
        #Docfrequency
        article_dic = defaultdict(list)
        keyterms = [] #save all distinct terms in questions
        
        #SentenceBOW
        bow_list = []
        
        #QuestionBOW
        que_list = []
        
        #SentenceLength
        sent_len = []
        
        #TotalLength
        total_len = 0
        
        #RightAnser
        right_answer = []
        
        qas = article['qa']
        sentences = article['sentences']
        for qa in qas:
            question = qa['question']
            newquestion = transform_text(question)
            #QuestionBOW
            que_list.append(get_BOW(newquestion))
            answer = qa['answer_sentence']
            right_answer.append(answer)
            
            keyterms.extend(newquestion)
        keyterms = set(keyterms)
        
        #save sentences' BOW in list sen_BOW
        sen_words = []
        for sent in sentences:
            sent = transform_text(sent)
            #Docfrequency
            sen_words.append(sent)
            
            #SentenceBOW
            bow_list.append(get_BOW(sent))
            
            #SentenceLength
            sent_len.append(len(sent))
            
            #TotalLength
            total_len += len(sent)
            
        
        #calculate doc frequency    
        for term in keyterms:
            for i,bow in enumerate(sen_words):
                if term in bow:
                    article_dic[term].append(i)
                    
        #Docfrequency
        question_list.append(article_dic)
        #SentenceBOW
        total_sentence_bow.append(bow_list)
        #QuestionBOW
        total_question_bow.append(que_list)
        #SentenceLength
        sent_lengthes.append(sent_len)
        #AverageLength
        avg_lengthes.append(float(total_len)/len(sentences))
        #
        answer_id.append(right_answer)
        
    return question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id

train,test,dev = input_data()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
punctuations = [',','\'\'','?','\'','.','%','(',')',';','``']
question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id_list= get_Docfrequency_SentenceBOW(train)

In [23]:
import math
def BM25(articles_index,k1,k2,b):
    total_queries = len(total_question_bow[articles_index])
    count = 0
    correct_id = []
    for index in range(len(total_question_bow[articles_index])):
        answer_id = answer_id_list[articles_index][index]
        guess_id = find_max_score_sentence(articles_index,index,k1,k2,b)
        if answer_id == guess_id:
            correct_id.append(index)
            count += 1   
    accurancy = float(count)/total_queries  
    return correct_id,accurancy

def find_max_score_sentence(articles_index,index,k1,k2,b):

    query_dict = total_question_bow[articles_index][index]
    max_score = 0
    guess_sentence = 0
    for index in range(len(total_sentence_bow[articles_index])):     
        score = 0  
        sentence_dict = total_sentence_bow[articles_index][index]
        for word in query_dict:
            document_fre_list = question_list[articles_index].get(word,None)
            
            
            N = len(total_sentence_bow[articles_index])
            n_qi = 0
            if document_fre_list != None:
                n_qi = len(document_fre_list)
            else:
                n_qi = 0
            fi = sentence_dict.get(word,0)
            qfi = query_dict.get(word,0)
            dl = sent_lengthes[articles_index][index]
            avgdl = avg_lengthes[articles_index]
            
            K = k1*(1-b+b*(float(dl)/avgdl)) 
            W = math.log((N-n_qi+0.5)/(n_qi+0.5))
            R = (fi*(k1+1))/(fi+K)*qfi*(k2+1)/(qfi+k2)
            score += W*R
        if score > max_score:
            max_score = score
            guess_sentence = index
    return guess_sentence

In [24]:
accuracy = 0.0

k1_list = [1.1]
k2_list = [0]
b_list = [0.18]

correct = []

test_length = len(train)
for k1 in k1_list:
    for k2 in k2_list:
        for b in b_list:
            accuracy = 0.0
            for i in range(0,test_length):
                correct_id,i_accuracy = BM25(i,k1,k2,b)
                accuracy += i_accuracy
                correct.append(correct_id)
            average_accuracy = accuracy/test_length
            print "Accuracy of BM25 with k1:",k1," k2:",k2," b:",b
            print average_accuracy

Accuracy of BM25 with k1: 1.1  k2: 0  b: 0.18
0.683639289744


In [ ]:
# Accuracy of BM25 with k1: 1.1  k2: 0  b: 0.18
# 0.683639289744

In [25]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import re
import sys

location_list = ['country','county']
number_list = ['version','size','msa','far','much','year','era','ration','years','time','many','population','district','large','percent','average','day','decade','big','long']
time_list = ['january','february','march','april','may','june','july','august','september','october','november','december','hours','minutes','miles','inches','foot','years','seasons','half']
name_list = ['university','name','center','president','denominations','denomination','film','broadcaster','pitcher','commentator']

def token(text):
    text = text[:-1]
    result = []
    combo = u''
    flag_upper = False
    x = re.split(r'("|\?|;|, |\(|\s|\))\s*', text)
    #print x
    for i,word in enumerate(x):
        if word != u'' and word != u' ':
            if word == u', ':
                word = u','
            if word == u'\u2013':
                word = u'-'
            if word.lower() in ['in','on','after','at','both']:
                word = word.lower()
            if word[0].isalpha():
                if flag_upper == False and word[0].isupper():
                    combo = word
                    flag_upper = True
                elif flag_upper == False and word[0].islower():
                    result.append(word)
                elif flag_upper == True and word[0].isupper():
                    combo = combo + u' ' + word
                elif flag_upper == True and word[0].islower():
                    result.append(combo)
                    combo = u''
                    result.append(word)
                    flag_upper = False
            else:
                if flag_upper == True:
                    result.append(combo)
                    combo = u''
                    result.append(word)
                    flag_upper = False
                else:
                    result.append(word)
    if combo != u'':
        result.append(combo) 
    return result
def token_question(text):
    text = text[:-1]
    result = []
    combo = u''
    flag_upper = False
    x = re.split(r'("|\?|;|, |\(|\s|\))\s*', text)
    for i,word in enumerate(x):
        if i == 0 and word != u'':
            result.append(word)
        else:
            if word == u', ':
                word = u','
            if word != u'' and word != u' ':
                if word[0].isalpha():
                    if flag_upper == False and word[0].isupper():
                        combo = word
                        flag_upper = True
                    elif flag_upper == False and word[0].islower():
                        result.append(word)
                    elif flag_upper == True and word[0].isupper():
                        combo = combo + u' ' + word
                    elif flag_upper == True and word[0].islower():
                        result.append(combo)
                        combo = u''
                        result.append(word)
                        flag_upper = False
                else:
                    if flag_upper == True:
                        result.append(combo)
                        combo = u''
                        result.append(word)
                        flag_upper = False
                    else:
                        result.append(word)
    if combo != u'':
        result.append(combo) 
    return result
def transfer_pos_question(pos):
    new_pos = []
    for (word,wtype) in pos:
        if word.lower() == 'what' or word.lower() == 'what\'s':
            new_pos.append((word,'WHAT'))
        elif word.lower() == 'do' or word.lower() == 'does' or word.lower() == 'did':
            new_pos.append((word,'DO'))
        elif word.lower() in number_list:
            new_pos.append((word,'TIME'))
        elif word.lower() == 'is' or word.lower() == 'was' or word.lower() == 'are' or word.lower() == 'were' or word.lower() == 'be':
            new_pos.append((word,'BE'))
        elif word.lower() in location_list:
            new_pos.append((word,'LOC'))
        elif word.lower() == 'when':
            new_pos.append((word,'WHEN'))
        elif word.lower() == 'where':
            new_pos.append((word,'WHERE'))
        elif word.lower() == 'can':
            new_pos.append((word,'CAN'))
        elif word.lower() == 'how':
            new_pos.append((word,'HOW'))
        elif word.lower() == 'who' or word.lower() == 'whom' or word.lower() == 'whose'  or word.lower() == 'whos':
            new_pos.append((word,'WHO'))
        elif word.lower() == 'which':
            new_pos.append((word,'WHICH'))
        elif word.lower() in name_list:
            new_pos.append((word,'NAME'))
        elif word.lower() == 'define':
            new_pos.append((word,'DEFINE'))
        elif word.lower() == 'should':
            new_pos.append((word,'SHOULD'))
        elif word.lower() == 'why' or word.lower() == 'wy':
            new_pos.append((word,'WHY'))
        else:
            new_pos.append((word,wtype))
    return new_pos
def transfer_pos_sentence(pos):
    new_pos = []
    pattern_1 = re.compile(r'-[0-9]+s?')
    pattern_2 = re.compile(r'[0-9]+s?-')
    for (word,wtype) in pos:
        if word.lower() == 'and' or word.lower() == 'or':
            new_pos.append((word,'POSICC'))
        elif word.lower() == 'but' or word.lower() == 'without':
            new_pos.append((word,'NEGCC'))
        elif word.lower() == 'with':
            new_pos.append((word,'WITH'))
        elif word.lower() == 'during':
            new_pos.append((word,'DURING'))
        elif word.lower() == 'a' or word.lower() == 'an':
            new_pos.append((word,'A'))
        elif word == '"':
            new_pos.append((word,'"'))
        elif word.lower() in time_list or '$' in word or  pattern_1.search(word) or  pattern_2.search(word):
            new_pos.append((word,'TIME'))
        elif word == 'around':
            new_pos.append((word,'AROUND'))
        elif word == 'mm':
            new_pos.append((word,'CD'))   
        elif word == 'by':
            new_pos.append((word,'BY'))
        elif word[0].isupper():
            new_pos.append((word,'UPPER'))
        else:
            new_pos.append((word,wtype))
    return new_pos
def get_continuous_chunks(text,texttype):
    #text = text.encode('ascii','replace')
    t = copy.deepcopy(text)
    #print token(t)
    if texttype==0:
        pos = pos_tag(token(t))
        pos = transfer_pos_sentence(pos)
        grammar = r"""
                    BASICN:
                        {<UPPER>}
                        {<NN.*>}
                    J:
                        {<JJ.*><VBN>}
                        {<JJ.*><POSICC><JJ.*>}   
                        {<JJ.*>+}
                        {<NN.*><POS>}
                    N:
                        {<DT>?<BASICN>?<J>?<BASICN>+}
                        <\">{<A>?<J>?<BASICN>+}<\">
                    COMBON:
                        {(<N><,>)*<N><,>?<POSICC><N>}
                    NWC:
                        {<N><WITH><COMBON>}
                    """
    elif texttype==1:
        #print token_question(t)
        pos = pos_tag(token_question(t))
        pos = transfer_pos_question(pos)
        #print pos
        grammar = r"""
                    WHAT: 
                        {<WHAT>}
                        {<WHICH>}
                        {<DEFINE>}
                    WHO:
                        {<WHO>}
                        {<WHAT><BE>?<DT>?<JJ|RB>*<NAME>}
                        {<WHAT><JJ|RB>*<NN>+<NAME>}
                    WHEN:
                        {<WHICH><TIME>}
                        {<HOW><TIME>}
                        {<WHAT><BE>?<DT>?<JJ>?<NN>*<JJ>?<TIME>}
                        {<WHEN>}
                    WHERE:
                        {<WHERE>}
                        {<WHAT><LOC>}
                    HOW:
                        {<CAN>}
                        {<DO>}
                        {<SHOULD>}
                        {<WHY>}
                        {<HOW>}
                    
                    """
    elif texttype==2:
        #when
        pos = pos_tag(token(t))
        pos = transfer_pos_sentence(pos)
        #print pos
        grammar = r"""
                    C:
                        {<TIME><CD>?<,><CD>}
                        {<TIME><CD>?<,><TIME>}
                        {<CD><TIME>}
                        {<CD><\.><CD>}
                        {<AROUND><CD>+}
                        {<RB><TIME>+}   
                        {<JJ><TIME>+}
                        {<AROUND><TIME>+}
                        {<TIME>+<CD>+}
                        {<DT>?<TIME>+}
                        {<CD>+}      
                    COMBOC:
                        {<C><POSICC><C>}
                    """
    elif texttype==3:
        #when
        pos = pos_tag(token(t))
        pos = transfer_pos_sentence(pos)
        #print pos
        grammar = r"""
                    H:
                        {<RB><JJ><IN>?}
                        {<IN>?<JJ.*>*<NN.*>+}
                    HH:
                        {<H>+}
                    SEN:
                        {<.*>+}
                        }<VBD|BY|,|WITH|NEGCC|TO>+{
                    """
    elif texttype==4:
        #who
        pos = pos_tag(token(t))
        pos = transfer_pos_sentence(pos)
        #print pos
        grammar = r"""
                    UP:
                        {<DT>?<UPPER>}
                    SEN:
                        {<.*>+}
                        }<VB.*|BY|,|WITH>+{
                    """
    elif texttype == 5:
        #cannot find
        pos = pos_tag(token(t))
        pos = transfer_pos_sentence(pos)
        #print pos
        grammar = r"""
                    SEN:
                        {<.*>+}
                        }<DURING|,|WITH|NEGCC>+{
                    """
    cp = nltk.RegexpParser(grammar) 
    result = []
    poss = copy.deepcopy(pos)
    tree = cp.parse(pos)
    #record the position of pos
    flag = 0
    for subtree in tree.subtrees():
        if subtree.label() != 'S':
            phrase = u''
            for word,pos in subtree.leaves():
                if word == ',' or word == ')':
                    phrase = phrase + word
                else:
                    if len(phrase) > 0:
                        if (phrase[-1] == '(' or phrase[-1] == ')'):
                            phrase = phrase + word
                        else:
                            phrase = phrase + u' '
                            phrase = phrase + word
                    else:
                        phrase = phrase + u' '
                        phrase = phrase + word
            result.append((subtree.label(),phrase[1:]))         
    return poss,result

In [26]:
'''
NER
'''
time_word = ['million']

In [41]:
from nltk.tag import StanfordNERTagger
import re
def input_NER():
    stanford_dir = os.path.join('stanford-ner-2016-10-31')
    jarfile = os.path.join(stanford_dir,'stanford-ner.jar')
    modelfile = os.path.join(stanford_dir,'classifiers/english.all.3class.distsim.crf.ser.gz')
    return modelfile,jarfile
model,jar = input_NER()
st = StanfordNERTagger(model,jar)

def package_NER(ner_sentences):
    result_sentences = []
    for ner_sentence in ner_sentences:
        result_sentence = []
        perv_type = u'O'
        word = u''
        for index,(entity,etype) in enumerate(ner_sentence):
            if perv_type == u'O' and etype != u'O':
                perv_type = etype
                word = entity + u' '
            elif word != u'':
                if etype == u'O':
                    result_sentence.append((word[:-1],perv_type))
                    word = u''
                    perv_type = u'O'
                elif etype != perv_type:
                    result_sentence.append((word[:-1],perv_type))
                    word = entity + u' '
                    perv_type = etype
                elif etype == perv_type:
                    if entity in ['%']:
                        word = word[:-1] + entity + u' '
                    else:
                        word = word + entity + u' '
        if word != u'':
            result_sentence.append((word[:-1],perv_type))
        result_sentences.append(result_sentence)      
    return result_sentences

def parse_NER(ner_sentences):
    pattern_number = re.compile(r'([0-9]+|\%|\$)')
    result_sentences = []
    for ner_sentence in ner_sentences:
        result_sentence = []
        for index,(entity,etype) in enumerate(ner_sentence):
            entity.replace(u'\u2013',u'-')
            entity.replace(u'\u2014',u'-')
            entity.replace(u'\u2212',u'-')
            entity.replace(u'\u2044',u'%')
            if etype == u'O' and entity != u'':
                if pattern_number.search(entity) or entity in time_word:
                    result_sentence.append((entity,u'NUMBER'))
                elif index != 0 and entity[0].isupper():
                    result_sentence.append((entity,u'ORGANIZATION'))
                else:
                    result_sentence.append((entity,etype))
            else:
                result_sentence.append((entity,etype))
        result_sentences.append(result_sentence)
        
    return package_NER(result_sentences)

def extract_NER(parse_ner_sentence,mode):
    result = []
    if mode == 0:
        #PERSON
        for entity,etype in parse_ner_sentence:
            if etype == u'PERSON':
                result.append(entity)
    elif mode == 1:
        #NUMBER
        for entity,etype in parse_ner_sentence:
            if etype == u'NUMBER':
                result.append(entity)
    elif mode == 2:
        #LOCATION
        for entity,etype in parse_ner_sentence:
            if etype == u'LOCATION':
                result.append(entity)
    elif mode == 3:
        #ORGANIZATION
        for entity,etype in parse_ner_sentence:
            if etype == u'ORGANIZATION':
                result.append(entity)
    return result

In [28]:
def rank_rule_1(entity,query):
    #lower scores for content words also appear in the query
    count = 0
    length = len(entity)
    for word in entity:
        word = lemmatize(word.lower())
        if word not in stopwords:
            if word in query:
                count += 1
    if length == 0:
        score = 0
    else:
        score = 1 - float(count)/length
    return score

In [29]:
import string
def get_open_class_words(query):
    result = []
    for index in range(len(query)):
        if query[index] not in stopwords:
            if query[index] not in string.punctuation:
                result.append(query[index])
    return result

In [30]:
def rank_rule_3(answer_sentence,sentence,entity,query):
    #higher scores for closer distance between an entity and the headword
    #step 1: using a filter to extract "useful" open-class words
    results = get_open_class_words(query)
    sent = sentence
    original_sent = answer_sentence
    entity_loc = []
    query_loc = []
    for word in entity:
        if word in original_sent:
            entity_loc.append(original_sent.index(word))
    for q in results:
        if q in sent:
            query_loc.append(sent.index(q))
    min_dist = len(original_sent)
    if query_loc != []:
        for i in query_loc:
            for j in entity_loc:
                dist = abs(i - j)
                if dist < min_dist:
                    min_dist = dist
                    
    return 1 - float(min_dist)/len(original_sent)

In [65]:
total_count = 0.0
total_question = 0.0
c = 0.0
for index in range(len(train)):#range(len(train)):
    print "processing: ",index
    article = train[index]
    qas = article['qa']
    sentences = article['sentences']
    token_sentences = copy.deepcopy(sentences)
    for i in range(len(sentences)):
        token_sentences[i] = nltk.word_tokenize(token_sentences[i])
    ner_sentences = st.tag_sents(token_sentences)
    parse_ner_sentences = parse_NER(ner_sentences)
    total_question += len(correct[index])
    for i in correct[index]:
        answer_list = []
        qa = qas[i]
        flag = False
        text_question = qa['question']
        answer = qa['answer']
        answer_sentence_id = qa['answer_sentence']
        pos,result = get_continuous_chunks(text_question,1)
        #print pos
        if result != []:
            wtype,word = result[0]         
            #print '^'*100
            #print wtype
            if wtype == 'WHAT':               
                answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],3)
                if answer_list == []:
                    answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],0)
                    answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],1)
                    answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],2)
                if qa['answer'] in answer_list:
                    flag = True
                    total_count += 1
            elif wtype == 'WHEN':
                answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],1)
                if qa['answer'] in answer_list:
                    flag = True
                    total_count += 1
            elif wtype == 'WHO':
                answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],0)
                if answer_list == []:
                    answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],2)
                    answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],3)
                if qa['answer'] in answer_list:
                    flag = True
                    total_count += 1

            elif wtype == 'WHERE':
                answer_list = extract_NER(parse_ner_sentences[answer_sentence_id],2)
                if answer_list == []:
                    answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],0)
                    answer_list += extract_NER(parse_ner_sentences[answer_sentence_id],3)
                if qa['answer'] in answer_list:
                    flag = True
                    total_count += 1
            
#             if flag == True:
# #                 print '*'*100
# #                 print text_question
# #                 print wtype
#                 print '^'*100
#                 print text_question
#                 print wtype
#                 print sentences[answer_sentence_id]
#                 print parse_ner_sentences[answer_sentence_id]
#                 print 'A: ',answer
#                 print answer_list
#             elif wtype in ['WHAT','WHEN','WHO','WHERE'] and flag == False:
#                 print '^'*100
#                 print text_question
#                 print wtype
#                 print sentences[answer_sentence_id]
#                 print parse_ner_sentences[answer_sentence_id]
#                 print 'A: ',answer
#                 print answer_list
#         else:
#             print '^'*100

        if answer_list != []:
            query = copy.deepcopy(text_question)
            query = nltk.word_tokenize(query)
            #token the answer sentence and copy it for further usage
            answer_sentence = sentences[answer_sentence_id]
            answer_sentence = nltk.word_tokenize(answer_sentence)
            sentence = copy.deepcopy(answer_sentence)

            for query_index in range(len(query)):
                query[query_index] = lemmatize(query[query_index].lower())

            for sent_index in range(len(sentence)):
                sentence[sent_index] = lemmatize(sentence[sent_index].lower())

            scores_1 = []
            scores_3 = []
            scores = []
            if len(answer_list) == 1:
                answer = answer_list[0]
            else:
                for entity in answer_list:
                    entity = nltk.word_tokenize(entity) 
                    score1 = rank_rule_1(entity,query)
                    scores_1.append(score1)
                    #answer_sentence is the original version and sentence is preprocessed
                    score3 = rank_rule_3(answer_sentence,sentence,entity,query)
                    scores_3.append(score3)
                    w1 = 0.5
                    w3 = 1 - w1
                    total = w1 * score1 + w3 * score3
                    scores.append(total)
                answer = answer_list[scores.index(max(scores))]
                
            if qa['answer'] == answer:
                c += 1
#             else:
#                 print '*'*100
#                 print 'Question: ',text_question
#                 print 'lists: ',answer_list
#                 print 'scores: ',scores
#                 print 'pred: ',answer
#                 print 'label: ',qa['answer']
print c/total_question
print total_count/total_question

processing:  0
****************************************************************************************************
Question:  What does LP stand for when it comes to time capacity?
lists:  [u'LP', u'SP', u'EP']
scores:  [0.5, 0.9663865546218487, 0.9411764705882353]
pred:  SP
label:  long playing
****************************************************************************************************
Question:  What are common diameters found in phonograph records?
lists:  [u'LP', u'SP', u'EP']
scores:  [0.7983193277310925, 0.7647058823529411, 0.7394957983193278]
pred:  LP
label:  12", 10", 7"
****************************************************************************************************
Question:  What was the primary use of a phonographic disc record?
lists:  [u'20th', u'1880s', u'1920s\u2013by', u'1920s']
scores:  [0.9, 0.7125, 0.6625, 0.625]
pred:  20th
label:  music reproduction
****************************************************************************************************
Qu

In [ ]:
# 0.118628417938
# 0.279625626928